# ELECTRA for Information Extraction


### 1.1. Mount to drive


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/VBDI_NLP/models')

### 1.2. Config to tf v1.xx

In [6]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


***clone model from git***

In [ ]:
!git clone https://github.com/Cinnamon/electra_japanese.git

Cloning into 'electra_japanese'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395
Receiving objects: 100% (395/395), 1.09 MiB | 2.30 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [3]:
cd electra_japanese/

/content/drive/MyDrive/VBDI_NLP/models/electra_japanese


In [ ]:
!mkdir -p data/finetuning_data/squad
!mkdir -p data/models/

In [ ]:
!cp -r "/content/drive/MyDrive/VBDI_NLP/models/jbddata/jbddata_dev_data.json" "/content/drive/MyDrive/VBDI_NLP/models/electra_japanese/data/finetuning_data/squad/dev.json" 
!cp -r "/content/drive/MyDrive/VBDI_NLP/models/jbddata/jbddata_train_data.json" "/content/drive/MyDrive/VBDI_NLP/models/electra_japanese/data/finetuning_data/squad/train.json"

In [ ]:
# !mv dev-v2.0.json data/finetuning_data/squad/dev.json
# !mv train-v2.0.json data/finetuning_data/squad/train.json

In [ ]:
!cp /content/drive/MyDrive/VBDI_NLP/models/electra_japanese/electra_small/vocab.txt data/vocab.txt

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/VBDI_NLP/models/electra_japanese/electra_small', 'data/models/electra_small', copy_function = shutil.copy)

'data/models/electra_small'

### 2. Data Statistic

In [ ]:
import os
os.listdir("data/finetuning_data/squad")

['dev.json', 'train.json']

In [5]:
import json
from pprint import pprint
import numpy as np

def view_squad_info(subset = 'train', get_impossible_exp = False):
    with open("data/finetuning_data/squad/{}.json".format(subset), "r") as f:
      data = json.load(f)
    
    numOfParagraph = 0
    numOfQaPair = 0
    ContextLen = []
    queryLen = [] # Độ dài của các query
    ansLen = [] # Độ dài của các câu trả lời
    # YOUR CODE HERE
    for i in range(len(data['data'])):
        numOfParagraph += len(data['data'])
        for j in range(len(data['data'][i]['paragraphs'])):
            numOfQaPair += len(data['data'][i]['paragraphs'][j]['qas']) # Thống kê số cặp câu hỏi câu trả lời
            ContextLen.append(len(data['data'][i]['paragraphs'][j]['context'])) # Thống kê độ dài của context
            for k in range(len(data['data'][i]['paragraphs'][j]['qas'])): # Thống kê độ dài của query và answer
                queryLen.append(len(data['data'][i]['paragraphs'][j]['qas'][k]['question']))
                for ans in range(len(data['data'][i]['paragraphs'][j]['qas'][k]['answers'])):
                    ansLen.append(len(data['data'][i]['paragraphs'][j]['qas'][k]['answers'][ans]['text']))
    maxContextLen = max(ContextLen)
    avgQueLen = np.mean(queryLen)
    avgAnsLen = np.mean(ansLen)


    print("Phiên bản SQuAd là {}".format(data["version"]))
    print("Số văn bản trong dataset là {}".format(len(data["data"])))
    print("Mỗi văn bản có những key sau: {}".format(data["data"][0].keys()))
    print("Số đoạn văn trong dataset là: {}".format(numOfParagraph))
    print("Số cặp câu hỏi và trả lời trong dataset là: {}".format(numOfQaPair))
    print("Độ dài tối đa của một đoạn văn là: {}".format(maxContextLen))
    print("Độ dài trung bình của một câu hỏi là: {}".format(avgQueLen))
    print("Độ dài trung bình của một trả lời là: {}".format(avgAnsLen))
    
    print("------MỘT SỐ CẶP CÂU VÍ DỤ-----")
    pprint(data["data"][0]['paragraphs'][0]["qas"][0:2])
    print("-------------------------------")
    pprint(data["data"][-1]['paragraphs'][0]["qas"][0:2])

    if get_impossible_exp:
      for i in range(len(data["data"])):
        for j in range(len(data["data"][i]['paragraphs'])):
          for k in range(len(data["data"][i]['paragraphs'][j]["qas"])):
            if data["data"][i]['paragraphs'][j]["qas"][k]['is_impossible']:
              pprint(data["data"][i]['paragraphs'][j]["qas"][k])

In [6]:
view_squad_info(subset = 'train')

Phiên bản SQuAd là v1.1
Số văn bản trong dataset là 164
Mỗi văn bản có những key sau: dict_keys(['title', 'paragraphs'])
Số đoạn văn trong dataset là: 26896
Số cặp câu hỏi và trả lời trong dataset là: 3974
Độ dài tối đa của một đoạn văn là: 4828
Độ dài trung bình của một câu hỏi là: 7.674886763965778
Độ dài trung bình của một trả lời là: 17.559134373427277
------MỘT SỐ CẶP CÂU VÍ DỤ-----
[{'answers': [{'answer_start': 22, 'text': '平成29年9月22日'}],
  'id': '729b958f6c724f53a7162375634e9ff1',
  'is_impossible': False,
  'question': '公告日'},
 {'answers': [{'answer_start': 128, 'text': '旧松尾鉱山新中和処理施設で使用する電気'}],
  'id': '040370df81494853a1aba57722696336',
  'is_impossible': False,
  'question': '入札件名'}]
-------------------------------
[{'answers': [{'answer_start': 0, 'text': '令和2年5月15日'}],
  'id': '730cad66abd64bf1adb22eee4d9e28d3',
  'is_impossible': False,
  'question': '公告日'}]


In [7]:
view_squad_info(subset = 'dev', get_impossible_exp= False)

Phiên bản SQuAd là v2.0
Số văn bản trong dataset là 22
Mỗi văn bản có những key sau: dict_keys(['title', 'paragraphs'])
Số đoạn văn trong dataset là: 484
Số cặp câu hỏi và trả lời trong dataset là: 440
Độ dài tối đa của một đoạn văn là: 4872
Độ dài trung bình của một câu hỏi là: 6.7
Độ dài trung bình của một trả lời là: 17.581749049429657
------MỘT SỐ CẶP CÂU VÍ DỤ-----
[{'answers': [{'answer_start': 67, 'text': '平成30年2月2日'}],
  'id': '17db1e2672f14fad8070f1143d3160fe',
  'is_impossible': False,
  'question': '公告日'},
 {'answers': [{'answer_start': 134, 'text': '七尾国家石油ガス備蓄基地高圧電力購入(平成30年度)'}],
  'id': 'f24850c151d641cc8dd3a17cf9ee03be',
  'is_impossible': False,
  'question': '入札件名'}]
-------------------------------
[{'answers': [{'answer_start': 22, 'text': '令和2年4月24日'}],
  'id': 'a4da2d7c543c4c9c978240588a49ace5',
  'is_impossible': False,
  'question': '公告日'},
 {'answers': [{'answer_start': 95,
               'text': '自然力活用型坑廃水処理の実規模相当実証試験に関する嫌気反応槽の内容物混合・充填工事'}],
  'id': '1d7cbb3213e2

In [8]:
view_squad_info(subset = 'test', get_impossible_exp= False)

Phiên bản SQuAd là v2.0
Số văn bản trong dataset là 20
Mỗi văn bản có những key sau: dict_keys(['title', 'paragraphs'])
Số đoạn văn trong dataset là: 400
Số cặp câu hỏi và trả lời trong dataset là: 400
Độ dài tối đa của một đoạn văn là: 4504
Độ dài trung bình của một câu hỏi là: 6.7
Độ dài trung bình của một trả lời là: 17.752118644067796
------MỘT SỐ CẶP CÂU VÍ DỤ-----
[{'answers': [{'answer_start': 22, 'text': '平成30年1月18日'}],
  'id': 'df56e977ace442f492519e87e447f018',
  'is_impossible': False,
  'question': '公告日'},
 {'answers': [{'answer_start': 97, 'text': 'むつ小川原国家石油備蓄基地電力購入(平成30年度)'}],
  'id': '0cc1d716b2e2461fa9a022c99c89cea9',
  'is_impossible': False,
  'question': '入札件名'}]
-------------------------------
[{'answers': [{'answer_start': 22, 'text': '令和2年5月15日'}],
  'id': '3a23e97fc0244c76a273740e5f1a0237',
  'is_impossible': False,
  'question': '公告日'},
 {'answers': [{'answer_start': 92, 'text': '令和2年度海洋鉱物資源調査試料第1回室内分析'}],
  'id': '2641764b0cc6497dbc357623b6c8d25c',
  'is_imposs

## 3: Training

In [16]:
!pip install sentencepiece
!pip install neologdn

     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for neologdn: filename=neologdn-0.4-cp37-cp37m-linux_x86_64.whl size=188239 sha256=7b2e7e10952753b3ca1acc2b2b8f0d1d6c4d3be5ace88705abf4b7b4615013a0
  Stored in directory: /root/.cache/pip/wheels/b3/6f/d4/c132b4e7aef22019e307e7673d97010644c9c15f28c0d0b018
Successfully built neologdn


In [22]:
# YOUR CODE HERE
!python run_finetuning.py --data-dir data --model-name electra_small --hparams '{"task_names": ["squad"], "debug": false, "do_train": false, "do_eval": true, "model_size": "small", "max_seq_length": 512, "num_train_epochs": 30, "learning_rate": 3e-5 ,"doc_stride":128, "max_answer_length": 512, "vocab_size": 32000, "learning_rate": 3e-5}'
# !python3 run_finetuning.py --data-dir data --model-name electra_small --hparams '{"model_size": "small", "task_names": ["squad"], "eval_batch_size": 8, "beam_size": 20, "train_batch_size": 8}'
# '{"model_size": "small", "task_names": ["squad"], "eval_batch_size": 16, "beam_size": 20, "train_batch_size": 32}'
# YOUR CODE HERE

model dir :  data/models/electra_small/finetuning_models/squad_model
Config: model=electra_small, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir data
debug False
do_eval True
do_lower_case True
do_train False
doc_stride 128
double_unordered True
embedding_size 128
eval_batch_size 32
gcp_project None
init_checkpoint data/models/electra_small
iterations_per_loop 1000
joint_prediction True
keep_all_models True
layerwise_lr_decay 0.8
learning_rate 3e-05
log_examples False
max_answer_length 512
max_query_length 64
max_seq_length 512
model_dir data/models/electra_small/finetuning_models/squad_model
model_hparam_overrides {}
model_name electra_small
model_size small
n_best_size 20
n_writes_test 5
num_tpu_cores 1
num_train_epochs 30
num_trials 1
predict_batch_size 32
preprocessed_data_dir data/models/electra_small/finetuning_tfrecords/squad_tfrecords
qa_eval_file <built-in method format of str object at 0x7f61791b73b0>
qa_na_f

In [23]:
with open("data/models/electra_small/results/squad_results.txt", "r") as f:
  result_file = f.read()

print(result_file.replace(" - ", "\n"))

squad: HasAns_exact: 34.63
HasAns_f1: 87.44
HasAns_total: 335.00
NoAns_exact: 0.00
NoAns_f1: 0.00
NoAns_total: 105.00
best_exact: 32.50
best_exact_thresh: -33.24
best_f1: 66.82
best_f1_thresh: -28.84
exact: 26.36
f1: 66.58
total: 440.00

